In [ ]:
!pip install transformers datasets
!pip install -U transformers

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlo

In [ ]:
from google.colab import files
uploaded = files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import zipfile
import os

zip_path = "NLP_dataset.zip"
extract_path = "NLP_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Kiểm tra các file trong thư mục giải nén
for root, dirs, files in os.walk(extract_path):
    for file in files:
        print(os.path.join(root, file))


In [ ]:
import json
from datasets import Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

def load_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    examples = []
    for item in data:
        # Đảm bảo item là dict (tránh None hoặc dữ liệu hỏng)
        if not isinstance(item, dict):
            continue

        question = item.get('question')
        sparql = item.get('sparql_wikidata')

        # Chỉ tiếp tục nếu cả 2 trường là chuỗi (tránh None)
        if isinstance(question, str) and isinstance(sparql, str):
            question = question.strip()
            sparql = sparql.strip()
            if question and sparql:
                examples.append({'input_text': question, 'target_text': sparql})

    return examples

train_data = load_data(r'/content/NLP_dataset/train.json')  # sửa path đúng với bạn
test_data = load_data(r'/content/NLP_dataset/test.json')

In [ ]:
print(f"Số lượng câu hỏi hợp lệ: {len(train_data)}")
print(f"Số lượng câu hỏi hợp lệ: {len(test_data)}")

In [ ]:
!pip install --upgrade transformers

In [ ]:
import numpy as np
import torch
# 2. Chuyển sang Hugging Face Datasets
# Assuming train_data and test_data are already loaded from previous steps
# train_data = load_data(r'/content/NLP_dataset/train.json')
# test_data = load_data(r'/content/NLP_dataset/test.json')
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

# 3. Load model và tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 4. Tokenize
max_input_length = 128
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['input_text']
    targets = examples['target_text']

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        padding="max_length",
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            padding="max_length",
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# 5. Định nghĩa metric đánh giá
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Only apply argmax if predictions are logits
    preds = np.argmax(preds, axis=-1) if preds.ndim == 3 else preds

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in labels as we cannot decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    exact_matches = [int(pred.strip() == label.strip()) for pred, label in zip(decoded_preds, decoded_labels)]

    # Token-level Precision, Recall
    total_tp, total_fp, total_fn = 0, 0, 0
    for pred, label in zip(decoded_preds, decoded_labels):
        pred_tokens = set(pred.strip().split())
        label_tokens = set(label.strip().split())
        tp = len(pred_tokens & label_tokens)
        fp = len(pred_tokens - label_tokens)
        fn = len(label_tokens - pred_tokens)
        total_tp += tp
        total_fp += fp
        total_fn += fn

    # Handle case where total_tp + total_fp is zero
    precision = total_tp / (total_tp + total_fp + 1e-8) if (total_tp + total_fp) > 0 else 0.0
    # Handle case where total_tp + total_fn is zero
    recall = total_tp / (total_tp + total_fn + 1e-8) if (total_tp + total_fn) > 0 else 0.0
    # Handle case where precision + recall is zero
    f1 = 2 * precision * recall / (precision + recall + 1e-8) if (precision + recall) > 0 else 0.0

    accuracy = np.mean(exact_matches) if exact_matches else 0.0 # Handle empty exact_matches list


    return {
        "exact_match": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# 6. TrainingArguments – giảm batch size để tránh OOM
training_args = TrainingArguments(
    output_dir="./bart_kbqa",
    per_device_train_batch_size=16, # Giữ nguyên hoặc điều chỉnh nếu cần
    per_device_eval_batch_size=8,  # Giảm batch size cho evaluation
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=10,
    push_to_hub=False,
    report_to="none",
    ddp_find_unused_parameters=False,
    gradient_checkpointing=True, # Bật gradient checkpointing để tiết kiệm bộ nhớ
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 8. Train
trainer.train()

# 9. Evaluate
metrics = trainer.evaluate()
print(metrics)